# How do we create catalogs? 

We're often both busy and sleepy while observing, so entering details of individual targets one-by-one is an easy way for mistakes to happen. To remedy, this we can make target catalogs, where we've done the hard work of getting precise coordinates ahead of time, and can just select a target at the click of a button during the actual night!

In [1]:
from kosmoscraftroom.catalogs import *

## Make a catalog from a table of coordinates.

We can make a catalog out of an astropy Table, as long as it a columns of `names` and one with `SkyCoord` objects for the coordinates.

In [ ]:
names = ["A", "B", "C"]
coordinates = SkyCoord(ra=[1, 2, 3] * u.hourangle, dec=[-1, 0, 1] * u.deg)
table = Table(dict(names=names, sky_coordinates=coordinates))
table

In [3]:
tiny = TUICatalog("random-tiny-test")
tiny.from_table(table)

In [ ]:
tiny.table

## Make a catalog from a previous TUI catalog. 

There are some existing TUI catalogs floating around. We can load one into a TUI Catalog object, which might then be written out again or combined with other catalogs.

In [5]:
standards = TUICatalog(name="flux-calibrators")
standards.from_TUI("standards.tui")

In [ ]:
standards

In [ ]:
standards.table

## Make a catalog from an `exoatlas` population.

We can convert an `exoatlas` population, filtered down however we like, into a TUI catalog. 

In [ ]:
from exoatlas import *

# define some populations
e = TransitingExoplanets()
e.update_values("SPECULOOS-3b", distance=24.754 * u.pc)

In [9]:
nearby = e[e.distance < 30 * u.pc]
emission = e[e.emission_snr() > 20]
reflection = e[e.reflection_snr() > 5]

In [10]:
n = TUICatalog("nearby")
n.from_exoatlas(nearby)

e = TUICatalog("emission")
e.from_exoatlas(emission)

r = TUICatalog("reflection")
r.from_exoatlas(reflection)

In [ ]:
n, e, r

In [ ]:
n.table

## Write catalog files for TUI and humans.

Let' s make a different TUI catalog for each of our sub-samples. By loading them separately into TUI, they can have different colors. 

In [ ]:
for x, c in zip([n, e, r, standards], ["red", "orange", "yellow", "black"]):
    x.to_TUI(color=c)

In [ ]:
!head nearby.tui

Now, let's make a human-friendly catalog that smooshes all these entries together and sorted by RA. Because the human friendly catalog will have the names of the original catalogs included as extra columns, we'll be able to figure where we should look in TUI to select each one.

In [ ]:
combined = n + e + r + standards
combined.to_human_friendly()

In [ ]:
!head nearby+emission+reflection+flux-calibrators.txt

Now, we can load the `.tui` catalogs into TUI, print out the `.txt` file (with a monospace font), and have a great night observing!